### Deploy Result Analysis

In [338]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [339]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [340]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# data file parameter
file = "200 Classification"  # must be excel file
sheets = "Sheet3"  # "Sheet3"
start_first_column = 0  # start column location number

# classification for Part 1
threshold_num = 1

# word all usage in sent
word_use_num_max = 5
word_use_num_min = 1

In [341]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [342]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [343]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [344]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [345]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/\
6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
#df_youtube_sent_result.sort_values(by="search_string", key=lambda x:x.str.len(), ascending=True, inplace=True)
df_youtube_sent_result.drop(["word_index"], axis=1, inplace=True)
df_youtube_sent_result = df_youtube_sent_result.sample(107)
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
9,fazla sorun gibi değil aslında benim için iyi ...,181,186,yani pek fazla sorun gibi değil aslında benim ...,UGyGF6BzXTM,https://www.youtube.com/watch?v=UGyGF6BzXTM&t=...
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
50,bunun içinde bizim bütün,460,462,bunun içinde bizim bütün fularlarımızın içinde...,tdnuZzBifR4,https://www.youtube.com/watch?v=tdnuZzBifR4&t=...
97,gibi aynı şekilde sadece daha yeni ve daha,454,459,bu da yeni parça gördüğünüz gibi aynı şekilde ...,smqSkXg1LXI,https://www.youtube.com/watch?v=smqSkXg1LXI&t=...
100,peki yani siz en çok böyle izin,282,284,peki yani siz en çok böyle izin almadan hareke...,ZSX8Onpr7Qw,https://www.youtube.com/watch?v=ZSX8Onpr7Qw&t=...
...,...,...,...,...,...,...
15,olabilir mi olabilir tabii ki belki de bu benim,408,411,farklılığı olabilir mi olabilir tabii ki belki...,5jZCJvqswAU,https://www.youtube.com/watch?v=5jZCJvqswAU&t=...
37,dur bunu buraya bunu buraya mı,412,414,oldum dur bunu buraya bunu buraya mı alsam,0EC2P7aCb6Y,https://www.youtube.com/watch?v=0EC2P7aCb6Y&t=...
106,burada böyle hayır öyle bir şey olabilir mi as...,489,493,mesela yüzde doksan dokuz deyip burada böyle h...,GZG9nDbIgXM,https://www.youtube.com/watch?v=GZG9nDbIgXM&t=...
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...


In [346]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
9,fazla sorun gibi değil aslında benim için iyi ...,181,186,yani pek fazla sorun gibi değil aslında benim ...,UGyGF6BzXTM,https://www.youtube.com/watch?v=UGyGF6BzXTM&t=...,"[96, 111, 27, 14, 174, 31, 4, 28, 55, 10]"
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
50,bunun içinde bizim bütün,460,462,bunun içinde bizim bütün fularlarımızın içinde...,tdnuZzBifR4,https://www.youtube.com/watch?v=tdnuZzBifR4&t=...,"[97, 158, 148, 113]"
97,gibi aynı şekilde sadece daha yeni ve daha,454,459,bu da yeni parça gördüğünüz gibi aynı şekilde ...,smqSkXg1LXI,https://www.youtube.com/watch?v=smqSkXg1LXI&t=...,"[27, 130, 181, 37, 18, 101, 3, 18]"
100,peki yani siz en çok böyle izin,282,284,peki yani siz en çok böyle izin almadan hareke...,ZSX8Onpr7Qw,https://www.youtube.com/watch?v=ZSX8Onpr7Qw&t=...,"[66, 60, 119, 44, 9, 56, 140]"
...,...,...,...,...,...,...,...
15,olabilir mi olabilir tabii ki belki de bu benim,408,411,farklılığı olabilir mi olabilir tabii ki belki...,5jZCJvqswAU,https://www.youtube.com/watch?v=5jZCJvqswAU&t=...,"[129, 5, 129, 128, 33, 74, 8, 1, 31]"
37,dur bunu buraya bunu buraya mı,412,414,oldum dur bunu buraya bunu buraya mı alsam,0EC2P7aCb6Y,https://www.youtube.com/watch?v=0EC2P7aCb6Y&t=...,"[144, 26, 77, 26, 77, 13]"
106,burada böyle hayır öyle bir şey olabilir mi as...,489,493,mesela yüzde doksan dokuz deyip burada böyle h...,GZG9nDbIgXM,https://www.youtube.com/watch?v=GZG9nDbIgXM&t=...,"[38, 56, 17, 46, 0, 16, 129, 5, 149, 149]"
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"


In [347]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,fazla sorun gibi değil aslında benim için iyi ...,181,186,yani pek fazla sorun gibi değil aslında benim ...,UGyGF6BzXTM,https://www.youtube.com/watch?v=UGyGF6BzXTM&t=...,"[96, 111, 27, 14, 174, 31, 4, 28, 55, 10]"
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
2,bunun içinde bizim bütün,460,462,bunun içinde bizim bütün fularlarımızın içinde...,tdnuZzBifR4,https://www.youtube.com/watch?v=tdnuZzBifR4&t=...,"[97, 158, 148, 113]"
3,gibi aynı şekilde sadece daha yeni ve daha,454,459,bu da yeni parça gördüğünüz gibi aynı şekilde ...,smqSkXg1LXI,https://www.youtube.com/watch?v=smqSkXg1LXI&t=...,"[27, 130, 181, 37, 18, 101, 3, 18]"
4,peki yani siz en çok böyle izin,282,284,peki yani siz en çok böyle izin almadan hareke...,ZSX8Onpr7Qw,https://www.youtube.com/watch?v=ZSX8Onpr7Qw&t=...,"[66, 60, 119, 44, 9, 56, 140]"
...,...,...,...,...,...,...,...
102,olabilir mi olabilir tabii ki belki de bu benim,408,411,farklılığı olabilir mi olabilir tabii ki belki...,5jZCJvqswAU,https://www.youtube.com/watch?v=5jZCJvqswAU&t=...,"[129, 5, 129, 128, 33, 74, 8, 1, 31]"
103,dur bunu buraya bunu buraya mı,412,414,oldum dur bunu buraya bunu buraya mı alsam,0EC2P7aCb6Y,https://www.youtube.com/watch?v=0EC2P7aCb6Y&t=...,"[144, 26, 77, 26, 77, 13]"
104,burada böyle hayır öyle bir şey olabilir mi as...,489,493,mesela yüzde doksan dokuz deyip burada böyle h...,GZG9nDbIgXM,https://www.youtube.com/watch?v=GZG9nDbIgXM&t=...,"[38, 56, 17, 46, 0, 16, 129, 5, 149, 149]"
105,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"


In [348]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [349]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [350]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[129,
 133,
 144,
 145,
 149,
 154,
 157,
 159,
 162,
 163,
 164,
 165,
 168,
 50,
 182,
 183,
 185,
 187,
 67,
 75,
 85,
 103,
 105,
 106,
 117,
 127]

In [351]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,fazla sorun gibi değil aslında benim için iyi ...,181,186,yani pek fazla sorun gibi değil aslında benim ...,UGyGF6BzXTM,https://www.youtube.com/watch?v=UGyGF6BzXTM&t=...,"[96, 111, 27, 14, 174, 31, 4, 28, 55, 10]"
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
2,bunun içinde bizim bütün,460,462,bunun içinde bizim bütün fularlarımızın içinde...,tdnuZzBifR4,https://www.youtube.com/watch?v=tdnuZzBifR4&t=...,"[97, 158, 148, 113]"
3,gibi aynı şekilde sadece daha yeni ve daha,454,459,bu da yeni parça gördüğünüz gibi aynı şekilde ...,smqSkXg1LXI,https://www.youtube.com/watch?v=smqSkXg1LXI&t=...,"[27, 130, 181, 37, 18, 101, 3, 18]"
4,peki yani siz en çok böyle izin,282,284,peki yani siz en çok böyle izin almadan hareke...,ZSX8Onpr7Qw,https://www.youtube.com/watch?v=ZSX8Onpr7Qw&t=...,"[66, 60, 119, 44, 9, 56, 140]"
...,...,...,...,...,...,...,...
68,tabii ki yoksa hiç kimse,336,338,kazanılmıyor hal böyle olunca da vatandaşımız ...,0IQUDH43SbU,https://www.youtube.com/watch?v=0IQUDH43SbU&t=...,"[128, 33, 150, 41, 146]"
69,birkaç gün sonra tekrar saat,603,605,birkaç gün sonra tekrar saat veriyor,B_PW8vRv9Qo,https://www.youtube.com/watch?v=B_PW8vRv9Qo&t=...,"[191, 95, 42, 160, 195]"
70,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
71,içinde tam işte uzun,303,305,çıkıyor içinde tam işte uzun çıkmıyor para çık...,n5b8T70avq8,https://www.youtube.com/watch?v=n5b8T70avq8&t=...,"[158, 112, 108, 172]"


In [352]:
#df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)

In [353]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list 

In [354]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['olabilir',
 'kendi',
 'dur',
 'git',
 'asla',
 'vardı',
 'benimle',
 'baba',
 'özür',
 'mü',
 'dostum',
 'selam',
 'anne',
 'hadi',
 'bence',
 'gerçek',
 'pekala',
 'oh',
 'eğer',
 'doğru',
 'efendim',
 'et',
 'kötü',
 'gece',
 'gerek',
 'devam']

In [355]:
len(not_in_sent_word)

26